<a href="https://colab.research.google.com/github/Balestrauser/Python-Learning-Log/blob/main/Sinum_Project_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import plotly.express as px

# --- 1. CONFIGURAÇÃO INICIAL ---

# Cotação do Dólar (1 USD = 5.27 BRL)
COTACAO_DOLAR_REAL = 5.27

# Orçamento Fictício Total para o Fundo Global
ORCAMENTO_TOTAL_BRL = 50_000_000_000  # R$ 50 Bilhões

# --- 2. DADOS DOS PAÍSES (Extraídos das imagens) ---

# Vamos unificar os dados em Trilhões de USD para facilitar
# (Ex: 642 bilhões = 0.642 trilhões)

dados = {
    "País": [
        "Estados Unidos", "China", "Alemanha", "Japão", "Índia", "França",
        "Rússia", "Brasil", "Argentina", "Nigéria", "Paquistão", "África do Sul",
        "Singapura", "Dinamarca", "Cuba", "Haiti", "Venezuela"
    ],
    "PIB_USD_Trilhoes": [
        29.16, 18.75, 4.71, 4.07, 3.91, 2.63, 2.24, 2.17, 0.642,
        0.400, 0.413, 0.370, 0.424, 0.420, 0.1073, 0.0252, 0.196
    ],
    # --- 3. MODELO DE EQUIDADE: FATOR DE APLICABILIDADE ---
    # (1.2 = Alta, 1.0 = Média, 0.8 = Baixa)
    "Fator_Aplicabilidade": [
        1.2, 1.2, 1.2, 1.2, 1.0, 1.2, 1.2, 1.0, 1.2, 0.8,
        1.0, 1.0, 0.8, 1.2, 0.8, 0.8, 0.8
    ]
}

# Criar o DataFrame (a tabela de dados)
df = pd.DataFrame(dados)

# --- 4. CÁLCULOS DO MODELO DE EQUIDADE ---

# Converter PIB para Reais
df['PIB_BRL_Trilhoes'] = df['PIB_USD_Trilhoes'] * COTACAO_DOLAR_REAL

# Calcular o "PIB Ponderado" (PIB ajustado pelo Fator de Aplicabilidade)
# Este é o núcleo da lógica de EQUIDADE
df['PIB_Ponderado'] = df['PIB_BRL_Trilhoes'] * df['Fator_Aplicabilidade']

# Calcular o percentual de contribuição de cada país com base no PIB Ponderado
total_pib_ponderado = df['PIB_Ponderado'].sum()
df['Percentual_Contribuicao'] = df['PIB_Ponderado'] / total_pib_ponderado

# Calcular a contribuição final em Reais para o fundo de R$ 50 Bilhões
df['Contribuicao_Final_BRL'] = df['Percentual_Contribuicao'] * ORCAMENTO_TOTAL_BRL

# --- 5. EXIBIR OS RESULTADOS ---

# Formatar os números para melhor visualização na tabela
pd.options.display.float_format = '{:,.2f}'.format
print("--- Tabela de Contribuição do Fundo (Modelo de Equidade) ---")
print(df.sort_values(by="Contribuicao_Final_BRL", ascending=False).to_string())
print("\nOrçamento Total Arrecadado: R$ {:,.2f}".format(df['Contribuicao_Final_BRL'].sum()))


# --- 6. GERAR GRÁFICOS / DASHBOARD ---

print("\nGerando gráficos interativos...")

# Gráfico 1: Barras da Contribuição Final (Quem paga quanto?)
fig1 = px.bar(
    df.sort_values(by="Contribuicao_Final_BRL", ascending=False),
    x="País",
    y="Contribuicao_Final_BRL",
    title="Contribuição por País para o Fundo de 'Casas Verdes' (R$ 50 Bilhões)",
    labels={"Contribuicao_Final_BRL": "Contribuição (em Bilhões BRL)", "País": "País"},
    hover_data=["PIB_BRL_Trilhoes", "Fator_Aplicabilidade"]
)
fig1.show()

# Gráfico 2: Scatter Plot (PIB vs. Contribuição) - O MELHOR GRÁFICO
# Este gráfico mostra a "justiça" do modelo.
fig2 = px.scatter(
    df,
    x="PIB_BRL_Trilhoes",
    y="Contribuicao_Final_BRL",
    color="Fator_Aplicabilidade",  # Mostra os 3 grupos de equidade
    size="PIB_BRL_Trilhoes",       # Bolha maior para PIB maior
    hover_name="País",
    title="Análise de Equidade: Capacidade (PIB) vs. Contribuição Final",
    labels={
        "PIB_BRL_Trilhoes": "PIB Total (em Trilhões BRL)",
        "Contribuicao_Final_BRL": "Contribuição Final (BRL)",
        "Fator_Aplicabilidade": "Fator de Aplicabilidade"
    },
    log_x=True, # Usar escala de log no PIB para ver melhor os países menores
    log_y=True
)
fig2.show()

# Gráfico 3: Treemap (Visão proporcional do fundo)
fig3 = px.treemap(
    df,
    path=[px.Constant("Contribuições Totais"), 'Fator_Aplicabilidade', 'País'],
    values='Contribuicao_Final_BRL',
    color='PIB_BRL_Trilhoes',
    color_continuous_scale='Blues',
    title="Visão Proporcional das Contribuições (Tamanho = Contribuição, Cor = PIB)",
    hover_data=['Percentual_Contribuicao']
)
fig3.update_traces(textinfo="label+percent root")
fig3.show()

--- Tabela de Contribuição do Fundo (Modelo de Equidade) ---
              País  PIB_USD_Trilhoes  Fator_Aplicabilidade  PIB_BRL_Trilhoes  PIB_Ponderado  Percentual_Contribuicao  Contribuicao_Final_BRL
0   Estados Unidos             29.16                  1.20            153.67         184.41                     0.42       21,096,954,832.55
1            China             18.75                  1.20             98.81         118.57                     0.27       13,565,428,776.07
2         Alemanha              4.71                  1.20             24.82          29.79                     0.07        3,407,635,708.55
3            Japão              4.07                  1.20             21.45          25.74                     0.06        2,944,602,406.33
4            Índia              3.91                  1.00             20.61          20.61                     0.05        2,357,370,067.31
5           França              2.63                  1.20             13.86          16.63  